In [22]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

import cvxpy as cp
import cvxopt
from cvxopt import matrix

from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter('ignore')

In [37]:
def SDID(O, treat_units = [0], starting_time = 100):

    donor_units = []
    for i in range(O.shape[0]):
        if (i not in treat_units):
            donor_units.append(i)     
    
    ##Step 1, Compute regularization parameter
    
    D = O[:, 1:starting_time+1] - O[:, :starting_time]

    D_bar = np.mean(O[donor_units, :-1])

    z_square = np.mean((D - D_bar)**2)

    ##Step 2, Compute w^{sdid}
    Nco = len(donor_units)
    Ntr = len(treat_units)
    Tpre = starting_time
    Tpost = O.shape[1] - starting_time

    w = cp.Variable(Nco)
    w0 = cp.Variable(1)
    G = np.eye(Nco)
    A = np.ones(Nco)
    #G @ w >= 0
    #A.T @ w == 1

    mean_treat = np.mean(O[treat_units, :Tpre], axis = 0)

    prob = cp.Problem(cp.Minimize(cp.sum_squares(w0+O[donor_units, :Tpre].T @ w - mean_treat) + z_square * Tpre * cp.sum_squares(w)), [G @ w >= 0, A.T @ w == 1])
    prob.solve()
    #print("\nThe optimal value is", prob.value) 
    #print("A solution w is")
    #print(w.value)

    w_sdid = np.zeros(O.shape[0]) 
    w_sdid[donor_units] = w.value
    w_sdid[treat_units] = -1.0 / Ntr

    ##Step 3, Compute l^{sdid}
    l = cp.Variable(Tpre)
    l0 = cp.Variable(1)
    G = np.eye(Tpre)
    A = np.ones(Tpre)
    #G @ w >= 0
    #A.T @ w == 1

    mean_treat = np.mean(O[donor_units, Tpre:], axis = 1)
    #print(mean_treat.shape)

    prob = cp.Problem(cp.Minimize(cp.sum_squares(l0+O[donor_units, :Tpre] @ l - mean_treat)), [G @ l >= 0, A.T @ l == 1])
    prob.solve()
    #print("\nThe optimal value is", prob.value) 
    #print("A solution w is")
    #print(l.value)

    l_sdid = np.zeros(O.shape[1]) 
    l_sdid[:Tpre] = l.value
    l_sdid[Tpre:] = -1.0 / Tpost

    ##Step 4, Compute SDID estimator
    tau = w_sdid.T @ O @ l_sdid

    return tau

In [41]:
np.random.seed(1)
n = 50
T = 100
r = 5
mu = np.random.rand()
a = np.random.rand(n,1)
b = np.random.rand(1,T)
tau = 1

U = np.random.normal(loc=0, scale = 1, size = (n, r))
V = np.random.normal(loc = 0, scale = 1, size = (T, r))
M = 5 * U.dot(V.T) + np.random.normal(size = (n, T))

Ntr = int(n / 5)
Tpre = int(4*T / 5) 
treat_units = [i for i in range(Ntr)]
W = np.zeros((n, T))
W[treat_units, Tpre:] = 1

#print(treat_units)
tau_hat = SDID(M + W*tau, treat_units, Tpre) 
print(tau_hat)

1.0026997923361534


In [13]:
def compute_zeta_square(O, treat_units, starting_time):
    donor_units = []
    for i in range(O.shape[0]):
        if (i not in treat_units):
            donor_units.append(i)     
    
    ##Step 1, Compute regularization parameter
    
    D = O[:, 1:starting_time+1] - O[:, :starting_time]

    D_bar = np.mean(O[donor_units, :-1])

    z_square = np.mean((D - D_bar)**2)

    return z_square


def SDID_download(Y, treat_units = [0], starting_time = 100):
    
    s = treat_units
    t = starting_time

    Y_c = np.delete(Y, s, axis=0)
    Y_t = Y[s, :]

    Y_c_pre = Y_c[:, :t]
    Y_c_post = Y_c[:, t:]
    Y_t_pre = Y_t[:, :t]
    Y_t_post = Y_t[:, t:]

    sum_omega_YiT = omega_hat.T @ Y_c_post
    sum_lambda_YNt= lambda_hat.T @ Y_t_pre
    sum_omega_lambda_Yit = omega_hat.T @ Y_c_pre @ lambda_hat

    Yhat_sdid = sum_omega_YiT + sum_lambda_YNt - sum_omega_lambda_Yit
    #Yhat_sc = sum_omega_YiT
    #Yhat_did = Y_c_post.mean() + Y_t_pre.mean() - Y_c_pre.mean()

    tau_sdid = np.mean(Y_t_post - Yhat_sdid) 
    Y[s, t:] = Yhat_sdid
    return Yhat_sdid, tau_sdid

In [22]:
# Generate a random non-trivial quadratic program.
m = 15
n = 10
p = 5
np.random.seed(1)
P = np.random.randn(n, n)
P = P.T @ P
q = np.random.randn(n)
G = np.random.randn(m, n)
h = G @ np.random.randn(n)
A = np.random.randn(p, n)
b = np.random.randn(p)

# Define and solve the CVXPY problem.
x = cp.Variable(n)
#print(q.T @ q)
#print(x.shape, q.shape, P.shape)
prob = cp.Problem(cp.Minimize(cp.sum_squares(q.T @ x) + cp.sum_squares(x)))
#prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, P) + q.T @ x), [G @ x <= h, A @ x == b])
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution corresponding to the inequality constraints is")
print(prob.constraints[0].dual_value)


The optimal value is 0.0
A solution x is
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
A dual solution corresponding to the inequality constraints is


IndexError: list index out of range